In [14]:
import cutlass
import cutlass.cute as cute

from tract.categories import *
from tract.test_utils import *
from tract.layout_utils import *

## Constructing tuples and nested tuples

In [15]:
S = (2,2,2)
T = ((2,2),(5,5))
U = ((2,2),4,(9,(3,3)))
print("S =", S)
print("T =", T)
print("U =", U)

S = (2, 2, 2)
T = ((2, 2), (5, 5))
U = ((2, 2), 4, (9, (3, 3)))


In [3]:
S = (10,)
T = (10)
print("S =", S)
print("T =", T)

S = (10,)
T = 10


## Constructing layouts and morphisms

In [4]:
@cute.jit
def construct_example_layouts():
    A = cute.make_layout(shape=((4,4),4), stride=((16,1),4))
    B = cute.make_layout(shape=(8,64), stride=(64,1))
    C = cute.make_layout(shape=100, stride=2)
    print("A =", A)
    print("B =", B)
    print("C =", C)
construct_example_layouts()

A = ((4,4),4):((16,1),4)
B = (8,64):(64,1)
C = 100:2


In [5]:
f = Nest_morphism(domain=NestedTuple((4,4)), codomain=NestedTuple((4,2,4)), map=(1,3))
g = Nest_morphism(domain=NestedTuple((2,2,2,2)), codomain=NestedTuple((2,2,2,2)), map=(1,0,4,2))
h = Nest_morphism(domain=NestedTuple((16,(4,4),(4,4))), codomain=NestedTuple((16,4,4)), map=(1,2,0,3,0))
print(f"{'f':<4}= {f}")
print(f"{'g':<4}= {g}")
print(f"{'h':<4}= {h}")

f   = (4,4) --(1, 3)--> (4,2,4)
g   = (2,2,2,2) --(1, 0, 4, 2)--> (2,2,2,2)
h   = (16,(4,4),(4,4)) --(1, 2, 0, 3, 0)--> (16,4,4)


## Translating between tractable layouts and morphisms

In [6]:
@cute.jit
def test_is_tractable():
    A = cute.make_layout(shape=(2,2,2), stride=(1,2,4))
    B = cute.make_layout(shape=(2,2,2), stride=(1,7,4))
    A_is_tractable = is_tractable(A)
    B_is_tractable = is_tractable(B)
    print(f"A =", A)
    print(f"A is tractable: {A_is_tractable}")
    print(f"B =", B)
    print(f"B is tractable: {B_is_tractable}")
test_is_tractable()

A = (2,2,2):(1,2,4)
A is tractable: True
B = (2,2,2):(1,7,4)
B is tractable: False


In [7]:
@cute.jit
def construct_standard_representation():
    L = cute.make_layout(shape=(2,2,2), stride=(1,2,4))
    f_L = compute_Tuple_morphism(L)
    print(f"{'L':<5}= {L}")
    print(f"{'f_L':<5}= {f_L}")
construct_standard_representation()

L    = (2,2,2):(1,2,4)
f_L  = (2, 2, 2) --(2, 4, 6)--> (1, 2, 1, 2, 1, 2)


In [8]:
@cute.jit
def compute_layout_example():
    f = Nest_morphism(domain=NestedTuple(((5,5),8)), codomain=NestedTuple((5,8,5)), map=(1,3,2))
    L_f = compute_layout(f)
    print(f"{'f':<5}= {f}")
    print(f"{'L_f':<5}= {L_f}")
compute_layout_example()

f    = ((5,5),8) --(1, 3, 2)--> (5,8,5)
L_f  = ((5,5),8):((1,40),5)


## Composition

In [9]:
@cute.jit
def composition_example():
    A = cute.make_layout(shape=((4,4),4), stride=((16,1),4))
    B = cute.make_layout(shape=(8,64), stride=(64,1))
    B_o_A = cute.composition(B,A)
    print(f"{'A':<5}= {A}")
    print(f"{'B':<5}= {B}")
    print(f"{'B∘A':<5}= {B_o_A}")
composition_example()

A    = ((4,4),4):((16,1),4)
B    = (8,64):(64,1)
B∘A  = ((4,4),(2,2)):((2,64),(256,1))


In [10]:
f = Nest_morphism(domain=NestedTuple(((2,2),(2,2))), codomain=NestedTuple(((2,2,2),(2,2,2))), map=(3,2,6,5))
g = Nest_morphism(domain=NestedTuple(((2,2,2),(2,2,2))), codomain=NestedTuple((2,2,2,2)), map=(1,0,2,0,3,4))
g_o_f = f.compose(g)
print(f"{'f':<5}= {f}")
print(f"{'g':<5}= {g}")
print(f"{'g∘f':<5}= {g_o_f}")

f    = ((2,2),(2,2)) --(3, 2, 6, 5)--> ((2,2,2),(2,2,2))
g    = ((2,2,2),(2,2,2)) --(1, 0, 2, 0, 3, 4)--> (2,2,2,2)
g∘f  = ((2,2),(2,2)) --(2, 0, 4, 3)--> (2,2,2,2)


## Coalesce

In [11]:
@cute.jit 
def coalesce_example():
     A = cute.make_layout(shape = ((2,2),(2,2),(5,5)), stride = ((1,2),(16,32),(64,640)))
     coal_A = cute.coalesce(A)
     print(f"{'A':<7}= {A}")
     print(f"{'coal_A':<7}= {coal_A}")
coalesce_example()

A      = ((2,2),(2,2),(5,5)):((1,2),(16,32),(64,640))
coal_A = (4,20,5):(1,16,640)


In [12]:
@cute.jit
def relative_coalesce_example():
     A = cute.make_layout(shape = ((2,2),(3,3),(5,5)), stride = ((1,2),(4,12),(36,180)))
     S = ((2,2),9,25)
     coal_A_over_S = cute.coalesce(A,target_profile=S)
     print(f"{'A':<15}= {A}")
     print(f"{'S':<15}= {S}")
     print(f"{'coal_A_over_S':<15}= {coal_A_over_S}")
relative_coalesce_example()

A              = ((2,2),(3,3),(5,5)):((1,2),(4,12),(36,180))
S              = ((2, 2), 9, 25)
coal_A_over_S  = ((2,2),9,25):((1,2),4,36)


In [17]:
f = Nest_morphism(domain=NestedTuple((2,2,10,10)), codomain=NestedTuple((2,2,2,10,10)), map=(1,2,4,5))
coal_f = f.coalesce()
print(f"{'f':<7}= {f}")
print(f"{'coal_f':<7}= {coal_f}")

AttributeError: 'Nest_morphism' object has no attribute 'coalesce'

## Complement

In [ ]:
@cute.jit
def complement_example():
     A = cute.make_layout(shape = ((2,2),(2,2)), stride = ((8,2),(64,256)))
     comp_A = cute.complement(A,4096)
     print(f"{'A':<7}= {A}")
     print(f"{'comp_A':<7}= {comp_A}")
complement_example()

A      = ((2,2),(2,2)):((8,2),(64,256))
comp_A = (2,2,4,2,8):(1,4,16,128,512)


In [18]:
f = Nest_morphism(domain=NestedTuple((2,2)), codomain=NestedTuple((2,5,2,5)), map=(1,3))
comp_f = f.complement()
print(f"{'f':<7}= {f}")
print(f"{'comp_f':<7}= {comp_f}")

f      = (2,2) --(1, 3)--> (2,5,2,5)
comp_f = (5,5) --(2, 4)--> (2,5,2,5)


## Logical Division

In [21]:
@cute.jit
def logical_divide_example():
    A = cute.make_layout((64,32), stride = (32,1))
    B = cute.make_layout((4,4), stride = (1,64))
    quotient = cute.logical_divide(A,B)
    print(f"{'A':<9}= {A}")
    print(f"{'B':<9}= {B}")
    print(f"{'quotient':<9}= {quotient}")
logical_divide_example()

A        = (64,32):(32,1)
B        = (4,4):(1,64)
quotient = ((4,4),(16,8)):((32,1),(128,4))


## Logical Product

In [22]:
@cute.jit
def logical_product_example():
    A = cute.make_layout((3,10,10), stride = (200,1,20))
    B = cute.make_layout((2,2), stride = (1,2))
    product = cute.logical_product(A,B)
    print(f"{'A':<9}= {A}")
    print(f"{'B':<9}= {B}")
    print(f"{'product':<9}= {product}")
logical_product_example()

A        = (3,10,10):(200,1,20)
B        = (2,2):(1,2)
product  = ((3,10,10),(2,2)):((200,1,20),(10,600))
